In [19]:
import neuropythy as ny
import nibabel as nib
import numpy as np

import ipyvolume as ipv

In [4]:
# You need to configure neuropythy so that it knows what your
# HCP AWS S3 access key and secret are:
ny.config['hcp_credentials'] = ('<key>', '<secret>')
ny.config['hcp_auto_download'] = True
# Auto-download will work without these commands, but they let
# neuropythy find the cached files next time you start it up
# (without this, everything you download will be lost when
# your python process quits).
ny.config['hcp_auto_path'] = '~/hcp_data'

# There are also instructions in the neuropythy README.md file
# (see https://github.com/noahbenson/neuropythy/) on how to
# set this up in the ~/.npythyrc configuration file. Additional
# info is also in the neuropythy wiki under the configuration
# heading.

In [6]:
# Get a sample HCP subject:
sub = ny.hcp_subject(100206)
# (This yields a neuropythy Subject object)
sub

Subject(<100206>, <'s3://hcp-openaccess/HCP_1200/100206'>)

In [23]:
# sub.lh and sub.rh are the "native" (FreeSurfer) hemispheres;
# sub.hemis['lh_LR32k'] and sub.hemis['rh_LR32k'] are the
# HCP subject-aligned fs_LR hemispheres (with 32k resolution).
lh_hemi_native = sub.lh
rh_hemi_native = sub.rh
lh_hemi_LR32k = sub.hemis['lh_LR32k']
rh_hemi_LR32k = sub.hemis['rh_LR32k']

In [24]:
# Note that "properties" of the hemispheres are just data
# vectors that neuropythy keeps track of (these are from
# FreeSurfer or the HCP pipelines, generally).
lh_prop_dict = lh_hemi_native.properties
sorted(lh_prop_dict.keys())

['BA1_label',
 'BA1_weight',
 'BA2_label',
 'BA2_weight',
 'BA3a_label',
 'BA3a_weight',
 'BA3b_label',
 'BA3b_weight',
 'BA44_label',
 'BA44_weight',
 'BA45_label',
 'BA45_weight',
 'BA4a_label',
 'BA4a_weight',
 'BA4p_label',
 'BA4p_weight',
 'BA6_label',
 'BA6_weight',
 'DKT40_parcellation',
 'Desikan06_parcellation',
 'Deskian06_parcellation',
 'Destrieaux09_parcellation',
 'Destrieux09_parcellation',
 'MT_label',
 'MT_weight',
 'V1_label',
 'V1_weight',
 'V2_label',
 'V2_weight',
 'areal_distortion',
 'atlas_curvature',
 'atlas_label',
 'brodmann_area',
 'brodmann_area_wide',
 'convexity',
 'cortex_label',
 'curvature',
 'entorhinal_label',
 'entorhinal_weight',
 'index',
 'jacobian_norm',
 'label',
 'midgray_surface_area',
 'parcellation',
 'perirhinal_label',
 'perirhinal_weight',
 'pial_curvature',
 'pial_surface_area',
 'roi',
 'surface_area',
 'thickness',
 'thickness_uncorrected',
 'white_curvature',
 'white_surface_area']

In [12]:
# We can plot properties with the ny.cortex_plot() function.
# If you pass a color option, it can be a named property that
# neuropythy knows about, or it can be a vector of property
# values (one per vertex). (You can find the number of vertices
# on a hemisphere with `hemi.vertex_count`.)
# The model won't appear until you run it because you can't
# embed 3D plots in notebooks the way you can 2D plots.
ny.cortex_plot(sub.lh, surface='inflated', color='thickness',
               # pyplot arguments like cmap, vmin, and vmax work:
               cmap='hot', vmin=0, vmax=6)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…

In [25]:
# Load the CIFTI file:
cii_filename = '~/data/connectivityml/HCP_PTN1200/groupICA/groupICA_3T_HCP1200_MSMAll_d100.ica/melodic_IC.dscalar.nii'
cii_obj = ny.load(cii_filename)

In [26]:
cii_obj

In [27]:
# Split the CIFTI object into hemisphere/subvoxel data:
(lh_data, rh_data, subvox_data) = ny.hcp.cifti_split(cii_obj)

In [17]:
# These data should be (data-points x vertices)
lh_data.shape

(100, 32492)

In [21]:
# The 32492 size indicates this is a 32k LR hemisphere:
lh_hemi = sub.hemis['lh_LR32k']
rh_hemi = sub.hemis['rh_LR32k']

# We can make an ipyvolume figure to plot both hemispheres on:
fig = ipv.figure()
# Then plot each hemisphere using whichever ICA component we
# want to visualize:
ica_num = 0
ny.cortex_plot(lh_hemi, surface='very_inflated',
               color=lh_data[ica_num], cmap='hot', figure=fig)
ny.cortex_plot(rh_hemi, surface='very_inflated',
               color=rh_data[ica_num], cmap='hot', figure=fig)

Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), projectionMatrix=(1.0, 0.0…